In [1]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import sys
sys.path.append("../src")
from myproject.pipelines.preprocessing_pipeline import preprocessing_pipeline
from myproject.pipelines.clean_pipeline import cleaning_pipeline

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Classification Models")

<Experiment: artifact_location='mlflow-artifacts:/702206731938119972', creation_time=1764089993614, experiment_id='702206731938119972', last_update_time=1764089993614, lifecycle_stage='active', name='Classification Models', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [2]:
df = pd.read_csv("/home/jakub/Pulpit/Credit original/data/raw/train.csv")
X = df.drop(columns=['Credit_Score'])
y = df['Credit_Score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

clean = cleaning_pipeline()
prep = preprocessing_pipeline()

/tmp/ipykernel_3110/1789011870.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/jakub/Pulpit/Credit original/data/raw/train.csv")


In [3]:
forest_clf = Pipeline([
    ("clean", clean),
    ("prep", prep),
    ("forest_clf", RandomForestClassifier(random_state=42))
])

cv = StratifiedKFold(n_splits=5, shuffle=True ,random_state=42)

scoring = ["accuracy","f1_macro"]

cv_base = cross_validate(
    forest_clf,
    X_train,
    y_train,
    scoring=scoring,
    cv=cv,
    n_jobs=-1,
    return_train_score=False,
)

#metryki oceny cross validacji
cv_acc_mean_base = cv_base['test_accuracy'].mean()
cv_acc_std_base = cv_base['test_accuracy'].std()

cv_f1_mean_base = cv_base['test_f1_macro'].mean()
cv_f1_std_base = cv_base['test_f1_macro'].std()

print("           CROSS VALIDACJA")
print("|B|CV|Accuracy mean: ", cv_acc_mean_base)
print("|B|CV|Accuracy std: ", cv_acc_std_base)
print("--------------------------------------")
print("|B|CV|F1_macro mean: ", cv_f1_mean_base)
print("|B|CV|Accuracy mean: ", cv_f1_std_base)

#Trening na X_test
forest_clf.fit(X_train, y_train)

y_pred_base = forest_clf.predict(X_test)

test_acc_base = accuracy_score(y_test, y_pred_base)
test_f1_base  = f1_score(y_test, y_pred_base, average="macro")

print("           TEST")
print("|B|T|Accuracy: ", test_acc_base)
print("|B|T|F1_macro: ", test_f1_base)

print(classification_report(y_test, y_pred_base))

/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline

           CROSS VALIDACJA
|B|CV|Accuracy mean:  0.776075
|B|CV|Accuracy std:  0.0029807717121577743
--------------------------------------
|B|CV|F1_macro mean:  0.7601605772074838
|B|CV|Accuracy mean:  0.0033318924660081105


/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


           TEST
|B|T|Accuracy:  0.78535
|B|T|F1_macro:  0.7707467350291181
              precision    recall  f1-score   support

        Good       0.73      0.71      0.72      3566
        Poor       0.78      0.79      0.79      5799
    Standard       0.80      0.81      0.81     10635

    accuracy                           0.79     20000
   macro avg       0.77      0.77      0.77     20000
weighted avg       0.79      0.79      0.79     20000



In [4]:
# with mlflow.start_run(run_name="Random_Forest_baseline"):
#     metrics_cv = {
#         "cv_accuracy_mean_b": float(cv_acc_mean_base), 
#         "cv_accuracy_std_b": float(cv_acc_std_base), 
#         "cv_f1_mean_b": float(cv_f1_mean_base),
#         "cv_f1_std_b": float(cv_f1_std_base)
#     }
#     mlflow.log_metrics(metrics_cv)
    
#     metrics_t = {
#         "accuracy_base": float(test_acc_base),
#         "f1_macro_base": float(test_f1_base),
#     }
    
#     mlflow.log_metrics(metrics_t)

In [5]:
param_distribution = {
    "forest_clf__n_estimators": [200, 300, 400, 500],
    "forest_clf__max_depth": [10, 15, 20, 25, None],
    "forest_clf__min_samples_leaf": [1, 2, 3, 5, 10],
    "forest_clf__min_samples_split": [2, 5, 10, 15],
    "forest_clf__max_features": ["sqrt", "log2", 0.5],
    "forest_clf__bootstrap": [True],
    "forest_clf__class_weight": [None, "balanced"],
}

random_search = RandomizedSearchCV(
    estimator=forest_clf,
    param_distributions=param_distribution,
    n_iter=30,
    scoring="f1_macro",
    n_jobs=-1,
    cv=cv,
    random_state=42,
    verbose=2,
    return_train_score=False
)

random_search.fit(X_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


[CV] END forest_clf__bootstrap=True, forest_clf__class_weight=balanced, forest_clf__max_depth=25, forest_clf__max_features=log2, forest_clf__min_samples_leaf=3, forest_clf__min_samples_split=5, forest_clf__n_estimators=300; total time=  57.4s


/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


[CV] END forest_clf__bootstrap=True, forest_clf__class_weight=balanced, forest_clf__max_depth=25, forest_clf__max_features=log2, forest_clf__min_samples_leaf=3, forest_clf__min_samples_split=5, forest_clf__n_estimators=300; total time=  57.7s


...

/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


[CV] END forest_clf__bootstrap=True, forest_clf__class_weight=balanced, forest_clf__max_depth=25, forest_clf__max_features=log2, forest_clf__min_samples_leaf=5, forest_clf__min_samples_split=5, forest_clf__n_estimators=400; total time= 1.1min


/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


[CV] END forest_clf__bootstrap=True, forest_clf__class_weight=balanced, forest_clf__max_depth=25, forest_clf__max_features=log2, forest_clf__min_samples_leaf=5, forest_clf__min_samples_split=5, forest_clf__n_estimators=400; total time=  53.7s


,estimator,Pipeline(step...m_state=42))])
,param_distributions,"{'forest_clf__bootstrap': [True], 'forest_clf__class_weight': [None, 'balanced'], 'forest_clf__max_depth': [10, 15, ...], 'forest_clf__max_features': ['sqrt', 'log2', ...], ...}"
,n_iter,30
,scoring,'f1_macro'
,n_jobs,-1
,refit,True
,cv,StratifiedKFo... shuffle=True)
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [6]:
best_forest = random_search.best_estimator_
best_params = random_search.best_params_
best_cv_score = random_search.best_score_

In [7]:
cv_tuned = cross_validate(
    best_forest,
    X_train,
    y_train,
    scoring=scoring,
    cv=cv,
    n_jobs=-1
)


cv_acc_mean_t = cv_tuned["test_accuracy"].mean()
cv_acc_std_t = cv_tuned["test_accuracy"].std()

cv_f1_mean_t = cv_tuned['test_f1_macro'].mean()
cv_f1_std_t = cv_tuned['test_f1_macro'].std()

print("TUNED | CV accuracy mean:", cv_acc_mean_t)
print("TUNED | CV accuracy std: ", cv_acc_std_t)
print("TUNED | CV f1 mean:      ", cv_f1_mean_t)
print("TUNED | CV f1 std:       ", cv_f1_std_t)

/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline

TUNED | CV accuracy mean: 0.76005
TUNED | CV accuracy std:  0.0027572064666977485
TUNED | CV f1 mean:       0.7426669321640568
TUNED | CV f1 std:        0.0031510166423074693


In [8]:
best_forest.fit(X_train, y_train)

y_pred_base = best_forest.predict(X_test)

test_acc_t = accuracy_score(y_test, y_pred_base)
test_f1_t  = f1_score(y_test, y_pred_base, average="macro")

print("           TEST")
print("|B|T|Accuracy: ", test_acc_t)
print("|B|T|F1_macro: ", test_f1_t)

print(classification_report(y_test, y_pred_base))

/home/jakub/.venvs/ml/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


           TEST
|B|T|Accuracy:  0.77005
|B|T|F1_macro:  0.7536950138055009
              precision    recall  f1-score   support

        Good       0.70      0.71      0.70      3566
        Poor       0.77      0.75      0.76      5799
    Standard       0.80      0.80      0.80     10635

    accuracy                           0.77     20000
   macro avg       0.75      0.75      0.75     20000
weighted avg       0.77      0.77      0.77     20000



In [9]:
with mlflow.start_run(run_name="Random_Forest_Tuned"):
    metrics_cv = {
        "cv_accuracy_mean_t": float(cv_acc_mean_t),
        "cv_accuracy_std_t": float(cv_acc_std_t),
        "cv_f1_mean_t": float(cv_f1_mean_t),
        "cv_f1_std_t": float(cv_f1_std_t),
    }
    mlflow.log_metrics(metrics_cv)

    metrics_t = {
        "accuracy_t": float(test_acc_t),
        "f1_macro_t": float(test_f1_t),
    }
    mlflow.log_metrics(metrics_t)

🏃 View run Random_Forest_Tuned at: http://127.0.0.1:5000/#/experiments/702206731938119972/runs/fbd47d1c7dbc4b43a83d219bab55c50f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/702206731938119972
